!virtualenv my_env

!virtualenv -p python3 my_env

!source my_env/bin/activate

!gcloud services enable vision.googleapis.com

!export PROJECT_ID=$(gcloud config get-value core/project)

!gcloud iam service-accounts create my-vision-sa \
  --display-name "my vision service account"

!gcloud iam service-accounts keys create ~/key.json \
  --iam-account my-vision-sa@${PROJECT_ID}.iam.gserviceaccount.com

!export GOOGLE_APPLICATION_CREDENTIALS=~/key.json

#verify if Install the Cloud Vision client library for Python
!pip3 freeze | grep google-cloud-vision #google-cloud-vision==2.0.0

!pip3 install -U pip google-cloud-vision

#Confirm the client library can be imported without issue like the below, and then you're ready to use the Vision API from real code!
!python3 -c "import google.cloud.vision"

7. Perform Label Detection
One of the Vision API's basic features is to identify objects or entities in an image, known as label annotation. Label detection identifies general objects, locations, activities, animal species, products, and more. The Vision API takes an input image and returns the most likely labels which apply to that image. It returns the top-matching labels along with a confidence score of a match to the image.

In this example, you will perform label detection on an image of a street scene in Shanghai. To do this, copy the following Python code into your IPython session (or drop it into a local file such as label_detect.py and run it normally):

In [9]:
PROJECT=!gcloud config list project --format "value(core.project)"
print("Your current GCP Project Name is: ", PROJECT)

Your current GCP Project Name is:  ['fml-gcp-project-01']


In [1]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-samples-data/vision/using_curl/shanghai.jpeg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.label_detection(image=image)

print('Labels (and confidence score):')
print('=' * 30)
for label in response.label_annotations:
    print(label.description, '(%.2f%%)' % (label.score*100.))

ModuleNotFoundError: No module named 'google.cloud'

8. Perform Text Detection
Text detection performs Optical Character Recognition (OCR). It detects and extracts text within an image with support for a broad range of languages. It also features automatic language identification.

In this example, you will perform text detection on an image of an Otter Crossing. Copy the following snippet into your IPython session (or save locally as text_dectect.py):

In [ ]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/otter_crossing.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.text_detection(image=image)

for text in response.text_annotations:
    print('=' * 30)
    print(text.description)
    vertices = ['(%s,%s)' % (v.x, v.y) for v in text.bounding_poly.vertices]
    print('bounds:', ",".join(vertices))

9. Perform Landmark Detection
Landmark detection detects popular natural and man-made structures within an image.

In [ ]:
from __future__ import print_function
from google.cloud import vision

image_uri = 'gs://cloud-vision-codelab/eiffel_tower.jpg'

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = image_uri

response = client.landmark_detection(image=image)

for landmark in response.landmark_annotations:
    print('=' * 30)
    print(landmark)

10. Perform Emotional Face Detection
Facial features detection detects multiple faces within an image along with the associated key facial attributes such as emotional state or wearing headwear.

In this example, you will detect the likelihood of emotional state from four different emotional likelihoods including: joy, anger, sorrow, and surprise.

To perform emotional face detection, copy the following Python code into your IPython session (or save locally as face_dectect.py):

In [ ]:
from __future__ import print_function
from google.cloud import vision

uri_base = 'gs://cloud-vision-codelab'
pics = ('face_surprise.jpg', 'face_no_surprise.png')

client = vision.ImageAnnotatorClient()
image = vision.Image()

for pic in pics:
    image.source.image_uri = '%s/%s' % (uri_base, pic)
    response = client.face_detection(image=image)

    print('=' * 30)
    print('File:', pic)
    for face in response.face_annotations:
        likelihood = vision.Likelihood(face.surprise_likelihood)
        vertices = ['(%s,%s)' % (v.x, v.y) for v in face.bounding_poly.vertices]
        print('Face surprised:', likelihood.name)
        print('Face bounds:', ",".join(vertices))

https://github.com/googlecodelabs/cloud-vision-python


12. Additional Resources
In addition to the source code for the four examples you completed in this codelab, below are additional reading material as well as recommended exercises to augment your knowledge and use of the Vision API with Python.

Learn More
Cloud Vision API documentation: cloud.google.com/vision/docs
Cloud Vision API home page & live demo: cloud.google.com/vision
Vision API label detection/annotation: cloud.google.com/vision/docs/labels
Vision API facial feature recognition: cloud.google.com/vision/docs/detecting-faces
Vision API landmark detection: cloud.google.com/vision/docs/detecting-landmarks
Vision API optical character recognition (OCR): cloud.google.com/vision/docs/ocr
Vision API "Safe Search": cloud.google.com/vision/docs/detecting-safe-search
Vision API product/corporate logo detection: cloud.google.com/vision/docs/detecting-logos
Python on Google Cloud Platform: cloud.google.com/python
Google Cloud Python client: googlecloudplatform.github.io/google-cloud-python
Codelab open source repo: github.com/googlecodelabs/cloud-vision-

Additional Study
Now that you have some experience with the Vision API under your belt, below are some recommended exercises to further develop your skills:

You've built separate scripts demoing individual features of the Vision API. Combine at least 2 of them into another script. For example, add OCR/text recognition to the first script that performs label detection (label_detect.py). You may be surprised to find there is text on one of the hats in that image!

Instead of our random images available on Google Cloud Storage, write a script that uses one or more of your images on your local filesystem. Another similar exercise is to find images online (accessible via http://).
Same as #2, but with local images on your filesystem. Note that #2 may be an easier first step before doing this one with local files.

Try non-photographs to see how the API works with those.

Migrate some of the script functionality into a microservice hosted on Google Cloud Functions, or in a web app or mobile backend running on Google App Engine.

If you're ready to tackle that last suggestion but can't think of any ideas, here are a pair to get your gears going:

Analyze multiple images in a Cloud Storage bucket, a Google Drive folder (use the Drive API), or a directory on your local computer. Call the Vision API on each image, writing out data about each into a Google Sheet (use the Sheets API) or Excel spreadsheet. (NOTE: you may have to do some extra auth work as G Suite assets like Drive folders and Sheets spreadsheets generally belong to users, not service accounts.)

Some people Tweet images (phone screenshots) of other tweets where the text of the original can't be cut-n-pasted or otherwise analyzed. Use the Twitter API to retrieve the referring tweet, extract and pass the tweeted image to the Vision API to OCR the text out of those images, then call the Cloud Natural Language API to perform sentiment analysis (to determine whether it's positive or negative) and entity extraction (search for entities/proper nouns) on them. (This is optional for the text in the referring tweet.)
License

Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy Unrestricted